In [26]:
import pandas as pd
import numpy as np
import os 
import cv2
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pickle
from tqdm import tqdm
import random

In [27]:
def load_samples(filename):
    data = np.load(filename)
    benign, malicious = data['arr_0'], data['arr_1']
    benign = benign / 255.0
    malicious = malicious / 255.0
    return benign, malicious

In [28]:
data_file = '/Applications/ML projects/Success/Mal-cGAN/Numpy Dataset/imgs_256_revised.npz'
dataset = load_samples(data_file)
benign, malicious = dataset

In [29]:
samples = np.concatenate((benign, malicious), axis=0)
labels = (['benign'] * len(benign)) + (['malicious'] * len(malicious))

In [30]:
labels = np.array(labels)
print(samples.shape, labels.shape)

(146, 256, 256, 3) (146,)


In [31]:
shuffle = np.arange(samples.shape[0])
np.random.shuffle(shuffle)
samples = samples[shuffle]
labels = labels[shuffle]

In [32]:
train_size = 0.6
pivot = int(train_size * samples.shape[0])

In [33]:
bxtrain, bxtest = samples[:pivot], samples[pivot:]
bytrain, bytest = labels[:pivot], labels[pivot:]

In [34]:
bxtrain = bxtrain.reshape(bxtrain.shape[0], -1)
bxtest = bxtest.reshape(bxtest.shape[0], -1)

In [35]:
print(bxtrain.shape, bxtest.shape)

(87, 196608) (59, 196608)


In [36]:
param_grid = {
    'C':[0.1, 1, 10, 100],
    'gamma':[0.0001, 0.001, 0.1, 1],
    'kernel':['rbf', 'poly']
}

svc = svm.SVC(probability=True)
bmodel = GridSearchCV(svc, param_grid)

In [37]:
bhistory = bmodel.fit(bxtrain, bytrain)

In [39]:
bmodel_file = '/Applications/ML projects/Success/Mal-cGAN/Models/BlackBox-git.pkl'
with open(bmodel_file, 'wb') as file:
    pickle.dump(bmodel, file)

In [40]:
bmodel_file = '/Applications/ML projects/Success/Mal-cGAN/Models/BlackBox-git.pkl'
with open(bmodel_file, 'rb') as file:
    blackBox = pickle.load(file)

In [41]:
baccuracy = accuracy_score(blackBox.predict(bxtest), bytest)
baccuracy * 100

86.4406779661017

In [43]:
data_file = '/Applications/ML projects/Success/Mal-cGAN/Numpy Dataset/adv_imgs_256.npz'
dataset = load_samples(data_file)
abenign, amalicious = dataset
print(abenign.shape, amalicious.shape)

(61, 256, 256, 3) (170, 256, 256, 3)


In [44]:
random.shuffle(amalicious)

In [45]:
asamples = np.concatenate((abenign, amalicious), axis=0)

In [46]:
alabels = np.array((['benign'] * len(abenign)) + (['malicious'] * len(amalicious)))

In [47]:
shuffle = np.arange(asamples.shape[0])
np.random.shuffle(shuffle)
asamples = asamples[shuffle]
alabels = alabels[shuffle]

In [48]:
print(asamples.shape, alabels.shape)

(231, 256, 256, 3) (231,)


In [49]:
atrain_size = 0.60
apivot = int(atrain_size * asamples.shape[0])

In [50]:
axtrain, axtest = asamples[:apivot], asamples[apivot:]
aytrain, aytest = alabels[:apivot], alabels[apivot:]

In [51]:
axtrain = axtrain.reshape(axtrain.shape[0], -1)
axtest = axtest.reshape(axtest.shape[0], -1)

In [52]:
print(axtrain.shape)

(138, 196608)


In [53]:
ahistory = blackBox.fit(axtrain, aytrain)

In [54]:
amodel_file = '/Applications/ML projects/Success/Mal-cGAN/Models/BlackBox_Adv-git.pkl'
with open(amodel_file, 'wb') as file:
    pickle.dump(blackBox, file)

In [55]:
amodel_file = '/Applications/ML projects/Success/Mal-cGAN/Models/BlackBox_Adv-git.pkl'
with open(amodel_file, 'rb') as file:
    blackBox_adv = pickle.load(file)

In [56]:
aaccuracy = accuracy_score(blackBox_adv.predict(axtest), aytest)
aaccuracy * 100

94.6236559139785